# Residual Networks

---

In this notebook we will implement a very deep neural network using Residual Networks. 

Our framework of choice is Keras.

## Preliminaries

Let's start by loading the packages and dependencies we will need.

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import tensorflow as tf

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = f'res{stage}{block}_branch'
    bn_name_base = f'bn{stage}{block}_branch'
    
    # Retrieve filters
    filter_1, filter_2, filter_3 = filters
    
    X_shortcut = X
    
    # First component of the main path.
    X = Conv2D(
        filters=filter_1,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding='valid', 
        name=f'{conv_name_base}2a',
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2a')(X)
    X = Activation('relu')(X)
    
    # Second component of the main path
    X = Conv2D(
        filters=filter_2, 
        kernel_size=(f, f), 
        strides=(1, 1), 
        padding='same', 
        name=f'{conv_name_base}2b', 
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2b')(X)
    X = Activation('relu')(X)
    
    # Third component of the main path
    X = Conv2D(
        filters=filter_3, 
        kernel_size=(1, 1), 
        strides=(1, 1),
        padding='valid', 
        name=f'{conv_name_base}2c',
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2c')(X)
    
    # Final step: Add shorcut value to main path and activate with ReLU.
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [3]:
tf.reset_default_graph()

with tf.Session() as s:
    np.random.seed(1)
    previous_activations = tf.placeholder('float', [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    activations = identity_block(previous_activations, f=2, filters=[2, 4, 6], stage=1, block='a')
    
    s.run(tf.global_variables_initializer())
    
    out = s.run([activations], feed_dict={previous_activations: X, K.learning_phase(): 0})
    print(f'out: {out[0][1][1][0]}')

out: [0.19716819 0.         1.3561226  2.1713073  0.         1.3324987 ]


In [4]:
def convolutional_block(X, f, filters, stage, block, s=2):
    # Defining name basis
    conv_name_base = f'res{stage}{block}_branch'
    bn_name_base = f'bn{stage}{block}_branch'
    
    # Retrieve filters
    filter_1, filter_2, filter_3 = filters
    
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(
        filters=filter_1,
        kernel_size=(1, 1),
        strides=(s, s),
        padding='valid',
        name=f'{conv_name_base}2a',
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(
        filters=filter_2,
        kernel_size=(f, f), 
        strides=(1, 1), 
        name=f'{conv_name_base}2b',
        padding='same', 
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2b')(X)
    X = Activation('relu')(X)
    
    # Third component of main path
    X = Conv2D(
        filters=filter_3,
        kernel_size=(1, 1), 
        strides=(1, 1), 
        name=f'{conv_name_base}2c', 
        padding='valid', 
        kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=f'{bn_name_base}2c')(X)
    
    # Shortcut path
    X_shortcut = Conv2D(
        filters=filter_3,
        kernel_size=(1, 1), 
        strides=(s, s), 
        name=f'{conv_name_base}I', 
        padding='valid', 
        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=f'{bn_name_base}I')(X_shortcut)
    
    # Add both main and shortcut path and pass them through a ReLU activation.
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [6]:
tf.reset_default_graph()

with tf.Session() as s:
    np.random.seed(1)
    previous_activations = tf.placeholder('float', [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    activations = convolutional_block(previous_activations, f=2, filters=[2, 4, 6], stage=1, block='a')
    
    s.run(tf.global_variables_initializer())
    
    out = s.run([activations], feed_dict={previous_activations: X, K.learning_phase(): 0})
    print(f'out: {out[0][1][1][0]}')

out: [0.09018463 1.2348979  0.46822023 0.03671762 0.         0.65516603]
